# 单品 AAR 鲁棒库存优化（14天决策周期）

本笔记实现基于 Ben-Tal 等（2004）方法思想的“仿射可调鲁棒”库存优化（AAR）。
- 以14天作为一个决策周期（之前分析显示14天聚合误差更小）。
- 订购时点由二进制变量 `v_k` 决定；订单量采用仿射策略：`u_k = u_{k0} + \sum_{t<k} a_{k,t} (d_t - \hat d_t)`。
- 采用“箱型（Box）不确定集”：`d_t = \hat d_t \pm \Delta_t`，并通过场景极点（±）近似最坏情形。
- 额外提供名义/高低/随机场景模拟来评估策略表现。

若本地找不到“未来需求预测”CSV，将自动构造一个可复现示例数据（可在参数区替换为真实预测）。

In [ ]:
# 安装/导入依赖（若缺少将尝试安装）
import sys, subprocess, json, os

def ensure(pkg):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for p in ["pandas", "numpy", "pulp", "matplotlib", "seaborn"]:
    ensure(p)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp as pl

pd.set_option('display.max_columns', 120)
np.random.seed(42)

print("依赖加载完成")

In [ ]:
# 参数区：成本、不确定集、周期等
PRODUCT = "电解镍"  # 单品名（仅用于输出标注）
T_days = 14          # 决策周期长度（天）
K_periods = 8        # 优化期内的14天块数（例如 8*14=112天）

# 成本参数（可按需调整/接入真实值）
h = 1.0     # 单位期末库存持有成本（每14天块）
p = 10.0    # 单位缺货惩罚成本（backorder/未满足罚金）
c = 0.0     # 单位订购变动成本（每单位）
f = 100.0   # 固定订购成本（每次下单）
M = 1e6     # Big-M（足够大）

# 不确定集幅度（按14天聚合后的名义需求的比例设定区间）
# 例如 delta_ratio=0.2 表示 d_k ∈ [d_hat_k*(1-0.2), d_hat_k*(1+0.2)]
delta_ratio = 0.2

# 初始库存
I1 = 0.0


In [ ]:
# 读取/构造14天聚合需求预测

# 自动探测可能的预测输出文件名
candidate_files = [
    "all_future_predictions.csv",
]

# 也尝试从笔记本里之前输出常见命名
# 用户仓库中之前的notebook输出路径位于本机其他目录，这里仅尝试本目录

forecast_df = None
for name in candidate_files:
    if os.path.exists(name):
        try:
            df = pd.read_csv(name)
            if 'date' in df.columns and 'prediction' in df.columns:
                forecast_df = df.copy()
                break
        except Exception:
            pass

if forecast_df is None:
    # 回退：构造一个示例每日名义预测（均值 + 周期扰动）
    days = T_days * K_periods
    date_index = pd.date_range(start="2025-01-01", periods=days, freq="D")
    base = 1000
    season = 200*np.sin(np.arange(days)/7*2*np.pi)
    noise = np.random.normal(0, 50, size=days)
    pred = np.maximum(base + season + noise, 100)
    forecast_df = pd.DataFrame({
        'date': date_index,
        'prediction': pred
    })

# 14天聚合
forecast_df['date'] = pd.to_datetime(forecast_df['date'])
forecast_df = forecast_df.sort_values('date')
forecast_df['period'] = ((forecast_df['date'] - forecast_df['date'].min()).dt.days // T_days) + 1
agg = forecast_df.groupby('period', as_index=False)['prediction'].sum()

# 只取前 K_periods 个块
agg = agg.head(K_periods)

# 名义需求与不确定界
d_hat = agg['prediction'].to_numpy()
delta = delta_ratio * d_hat

print(f"检测到 {len(forecast_df)} 天预测，聚合为 {len(agg)} 个14天周期。")
agg.head()

In [ ]:
# 构建仿射可调鲁棒库存模型（简化版：极点场景法近似）
# 决策：
#  - v_k ∈ {0,1}: 第k块是否下单
#  - u_k0: 第k块名义下单量的常数项
#  - a_{k,t}: 仿射系数（t<k），表示对第t块需求偏差的线性响应
#  - y_k^+ / y_k^-: 期末库存与缺货（backorder），用分解变量逼近 |I_k^-|
# 库存动态：I_k = I_{k-1} + u_k - d_k
# 这里将 d_k = d_hat_k + z_k，z_k ∈ [-delta_k, +delta_k]，并以极点场景 z_k = ±delta_k 形成有限场景集合

K = len(d_hat)

# 生成极点场景：每个周期取 z_k ∈ { -delta_k, +delta_k }
# 为控制规模，采用“笛卡尔抽样的子集”：
#  - 全正、全负
#  - 单点翻转（每次只对一个k取相反号）
scenarios = []
scenarios.append(+delta.copy())            # 全正
scenarios.append(-delta.copy())            # 全负
for flip in range(K):                      # 单点翻转
    z = +delta.copy()
    z[flip] = -delta[flip]
    scenarios.append(z)
for flip in range(K):
    z = -delta.copy()
    z[flip] = +delta[flip]
    scenarios.append(z)

print(f"鲁棒极点场景数: {len(scenarios)}")

# 构建 MILP
m = pl.LpProblem("AAR_Robust_Inventory_14d", pl.LpMinimize)

# 变量
v = pl.LpVariable.dicts('v', list(range(1, K+1)), lowBound=0, upBound=1, cat=pl.LpBinary)
u0 = pl.LpVariable.dicts('u0', list(range(1, K+1)), lowBound=0)  # 常数项非负
# a_{k,t} 仅定义 t<k
A = {}
for k in range(1, K+1):
    for t in range(1, k):
        A[(k,t)] = pl.LpVariable(f"a_{k}_{t}", lowBound=None)  # 系数可正可负

# 对每个场景、每期的库存与缺货分解变量
I = {}
Ypos = {}
Yneg = {}
for s_idx, z in enumerate(scenarios):
    for k in range(1, K+1):
        I[(s_idx,k)] = pl.LpVariable(f"I_s{s_idx}_k{k}", lowBound=None)
        Ypos[(s_idx,k)] = pl.LpVariable(f"Ypos_s{s_idx}_k{k}", lowBound=0)
        Yneg[(s_idx,k)] = pl.LpVariable(f"Yneg_s{s_idx}_k{k}", lowBound=0)

# 为了避免仿射策略在极端下单，限定 u_k 的上界与Big-M和v_k关联
# 这里引入辅助变量 U_k(s) 表示该场景下的实际下单量，用线性表达式定义
U = {}
for s_idx, z in enumerate(scenarios):
    for k in range(1, K+1):
        U[(s_idx,k)] = pl.LpVariable(f"U_s{s_idx}_k{k}", lowBound=0)

# 目标：在所有场景上最坏情形成本最小化（minimize W），并对每个场景施加成本不超过 W
W = pl.LpVariable("W", lowBound=0)

# 成本分解：每个场景成本 = sum_k ( f*v_k + c*U_k + h*Ypos_k + p*Yneg_k )
# 注意：f*v_k 对所有场景相同，计入一次即可，但用“每场景成本≤W”时重复无害（上界更强）
for s_idx, z in enumerate(scenarios):
    scen_cost = []
    for k in range(1, K+1):
        scen_cost.append(f * v[k])
        scen_cost.append(c * U[(s_idx,k)])
        scen_cost.append(h * Ypos[(s_idx,k)])
        scen_cost.append(p * Yneg[(s_idx,k)])
    m += pl.lpSum(scen_cost) <= W, f"worst_cost_bound_s{s_idx}"

# 库存动态与 |I_k| 分解
for s_idx, z in enumerate(scenarios):
    # I_0 = I1（常数）
    m += I[(s_idx,1)] == I1 + U[(s_idx,1)] - (d_hat[0] + z[0]), f"inv_balance_s{s_idx}_k1"
    for k in range(2, K+1):
        # u_k = u0_k + sum_{t<k} a_{k,t} * z_t
        expr_u = u0[k] + pl.lpSum(A[(k,t)] * z[t-1] for t in range(1, k))
        m += U[(s_idx,k)] == expr_u, f"affine_u_s{s_idx}_k{k}"
        m += I[(s_idx,k)] == I[(s_idx,k-1)] + U[(s_idx,k)] - (d_hat[k-1] + z[k-1]), f"inv_balance_s{s_idx}_k{k}"
    # |I_k| 分解成 Ypos, Yneg，并且 I_k = Ypos - Yneg
    for k in range(1, K+1):
        m += I[(s_idx,k)] == Ypos[(s_idx,k)] - Yneg[(s_idx,k)], f"abs_split_s{s_idx}_k{k}"

# 订购触发与Big-M：U_k <= M * v_k；同时定义首期 u0_1 直接等于 U_s,k=1 的名义项（无历史z）
for s_idx, z in enumerate(scenarios):
    for k in range(1, K+1):
        m += U[(s_idx,k)] <= M * v[k], f"order_onoff_s{s_idx}_k{k}"

# 约束：k=1 的仿射下单量仅为常数项
for s_idx, z in enumerate(scenarios):
    m += U[(s_idx,1)] == u0[1], f"u_first_s{s_idx}"

# 目标
m += W

print("模型已构建。变量数：", len(m.variables()))

In [ ]:
# 求解模型
solver = pl.PULP_CBC_CMD(msg=False, timeLimit=60)
res = m.solve(solver)
print("Status:", pl.LpStatus[res])
print("最坏情形成本 W:", pl.value(W))

v_sol = np.array([pl.value(v[k]) for k in range(1, K+1)])
u0_sol = np.array([pl.value(u0[k]) for k in range(1, K+1)])

print("v(是否下单):", v_sol.astype(int))
print("u0(名义常数项):", np.round(u0_sol, 2))


In [ ]:
# 结果整理与可视化（名义场景）
# 名义场景取 z=0，则 u_k = u0_k；需求取 d_hat
U_nom = u0_sol.copy()
I_nom = np.zeros(K)
Ypos_nom = np.zeros(K)
Yneg_nom = np.zeros(K)

I_nom[0] = I1 + U_nom[0] - d_hat[0]
for k in range(1, K):
    I_nom[k] = I_nom[k-1] + U_nom[k] - d_hat[k]

Ypos_nom = np.maximum(I_nom, 0)
Yneg_nom = np.maximum(-I_nom, 0)

cost_nom = np.sum(f * v_sol + c * U_nom + h * Ypos_nom + p * Yneg_nom)
print(f"名义场景总成本: {cost_nom:.2f}")

fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
ax[0].bar(np.arange(1,K+1), d_hat, label='14天聚合需求(名义)')
ax[0].bar(np.arange(1,K+1), U_nom, alpha=0.7, label='下单量(名义仿射)')
ax[0].legend()
ax[0].set_ylabel('数量')

ax[1].plot(np.arange(1,K+1), I_nom, marker='o', label='期末库存(正为库存,负为缺口)')
ax[1].axhline(0, color='k', linewidth=1)
ax[1].legend()
ax[1].set_xlabel('第k个14天周期')
ax[1].set_ylabel('库存')
plt.tight_layout()
plt.show()

In [ ]:
# 简单场景模拟：全正/全负

def simulate(z):
    # 构造仿射下单量：u_k = u0_k + sum_{t<k} a_{k,t} z_t
    # 由于我们在求解阶段没有导出 A 系数到名义可视化，这里提取 A 解
    A_sol = np.zeros((K, K))
    for k in range(1, K+1):
        for t in range(1, k):
            A_sol[k-1, t-1] = pl.value(A[(k,t)]) if (k,t) in A else 0.0
    u = u0_sol.copy()
    for k in range(2, K+1):
        u[k-1] = u0_sol[k-1] + np.dot(A_sol[k-1, :k-1], z[:k-1])
    u = np.maximum(u, 0.0)

    I = np.zeros(K)
    I[0] = I1 + u[0] - (d_hat[0] + z[0])
    for k in range(1, K):
        I[k] = I[k-1] + u[k] - (d_hat[k] + z[k])
    y_pos = np.maximum(I, 0)
    y_neg = np.maximum(-I, 0)

    cost = np.sum(f * v_sol + c * u + h * y_pos + p * y_neg)
    return u, I, cost

z_plus = +delta.copy()
z_minus = -delta.copy()

u_p, I_p, cost_p = simulate(z_plus)
u_m, I_m, cost_m = simulate(z_minus)

print(f"全正场景成本: {cost_p:.2f}")
print(f"全负场景成本: {cost_m:.2f}")

In [ ]:
# 导出关键结果
result_df = pd.DataFrame({
    'period': np.arange(1, K+1),
    'd_hat_14d': d_hat,
    'order_flag_v': v_sol.astype(int),
    'u0_nominal': u0_sol,
    'I_nom_end': I_nom,
})
result_df['order_qty_nominal'] = np.where(result_df['order_flag_v']>0.5, result_df['u0_nominal'], 0.0)

print("订购计划（名义）预览：")
display(result_df.head(12))

out_csv = 'AAR_14days_plan.csv'
result_df.to_csv(out_csv, index=False)
print(f"已保存: {out_csv}")

## 备注
- 本实现采用“极点场景法”近似箱型不确定集下的 AAR 最坏情形。相较论文中的对偶/绝对值等价重整，这种方法更通用但可能带来更大的场景规模；这里通过“全正/全负 + 单点翻转”控制规模。
- 若要完全复刻 Ben-Tal et al. (2004) 的等价重整形式，需要将目标与约束按“常数项与不确定项分解”，对每个线性不等式引入左右 envelope 变量（类似文中式(18)-(25)），并对每个包含不确定变量的线性组合添加相应的上界变量与绝对值束缚。
- 实务上还可增加：最大库存容量、最小起订量、提前期、服务水平（CVaR）等扩展约束。

In [ ]:
# 随机多场景蒙特卡洛评估（围绕 Box 内均匀采样）
num_mc = 50
rng = np.random.default_rng(123)
mc_costs = []
for _ in range(num_mc):
    z = rng.uniform(low=-delta, high=+delta)
    u, I, cost = simulate(z)
    mc_costs.append(cost)

print(f"蒙特卡洛成本均值: {np.mean(mc_costs):.2f}, 标准差: {np.std(mc_costs):.2f}")
plt.figure(figsize=(8,4))
plt.hist(mc_costs, bins=12, alpha=0.7)
plt.axvline(np.mean(mc_costs), color='r', linestyle='--', label='均值')
plt.title('蒙特卡洛场景成本分布')
plt.legend()
plt.show()
